# The basics of data extraction

In [ ]:
#| label: python-setup-block
#| include: false

# Load packages

import timeit
import duckdb as db
import pandas as pd
import polars as pl
from statistics import mean, stdev
from random import seed

# Set seed

seed(121022)

# Connect to database

con = db.connect('../data/2020-anes/anes.db')

# Load the spatial extension for DuckDB
con.execute(
    '''
    INSTALL spatial; -- only need to run this once per connection
    LOAD spatial; -- only need to run this once per connection
    '''
)

```{r}
#| label: r-setup-block
#| include: false
# Load libraries

library(tictoc)
library(readr)
library(haven)
library(foreign)
library(readxl)
library(arrow)
```


The last chapter presented an overview of a principled workflow for data processing in a research project. I include a diagram of this workflow in @fig-workflow for your reference. This chapter will focus on the first part of the workflow: data extraction. When discussing data extraction, I am referring to the idea that you are taking raw data outputs from some source and are converting it into a usable tabular format. In this chapter, we are going to cover common data formats such as CSV and TSV files, Parquet files, and proprietary file types such as Microsoft's XLSX, STATA's DTA, and SPSS' SAV files.

```{mermaid}
%%| label: fig-workflow
%%| fig-cap: A principled workflow for data management
%%| fig-width: 6.5
%%{init: {'theme':'base', 'themeVariables':{'primaryColor':'#ffffff', 'primaryBorderColor': '#000000'}}}%%
flowchart TB
    subgraph s1[Data Extraction]
        direction TB
        A[Download CSV file] --> B[Create Participant ID Column]
        B --> C[Pull out identifiable data -- keep Participant ID column in each subset of data]
        C --> D[Store both subsets to two tables in a DuckDB file]
    end
    subgraph s2[Data Transformation]
        direction TB
        E[Clean data in stored DuckDB file and store as new tables -- do not overwrite original two tables]
    end
    subgraph s3[Data Loading]
        direction TB
        F[Load data to perform analyses from updated tables in DuckDB file]
    end
    subgraph s4[Data Distribution]
        direction TB
        G[Create a new DuckDB file with a copy of the cleaned tables used in the analyses and the table with de-identified original data]
        G --> H[Share new DuckDB file with copied tables along with SQL script used for the cleaning of the data when submitting to journal]
    end
    s1 --> s2
    s2 --> s3
    s3 --> s4
```



Though I recommend that you perform many of these tasks with `DuckDB`'s implementation of `SQL` goal of this chapter is to give you some examples of performing these steps with code in `Python`, and `R` as well. The objective is to encourage researchers to use this principled data processing strategy, even if I am unable to convince everyone to use `SQL`. I also will provide coding benchmarks to demonstrate the efficiency of `DuckDB`'s implementation of `SQL` relative to the common libraries in `Python`, and `R`.

## Data extraction

What is data extraction? Data extraction is a task that every researcher engages in. While many may think that it is a easy process, the reality is that in *some* cases data extraction is an easy process. When working with pre-compiled datasets, data extraction can certainly be an easy process. You can download some file and then load it into your statistical software of choice. However, even when you have a pre-compiled dataset, this process can still be quite challenging.

As the scale and the number of types of data that social scientists consider for their projects increases, data extraction steps are less standard and are orders of magnitude more complicated. In this chapter, we are focusing on data sets that have already been compiled, are in a tabular format, and are relatively small (thousands of rows and a couple of hundred columns). However, when we are working with raw data that are not in tabular format already -- such as reading text from PDF files -- or contains millions of observations for thousands of variables, data extraction is an extremely difficult process and choosing a tool that is responsive to scalability and facilitate documentation are imperative. The next chapter will provide examples of some of these more complicated features. However, the remainder of this chapter will be focused on providing simple and common examples of the data that social scientists will want to extract.

## Extracting data from Separated-Value files

Separated-value files are quite common for researchers. These files refer to those where we separate the different cells (or pieces of data) by some delimiter. The most common that we interact with are CSV files which refer to a separated-value file where the cells are separated by commas, `,`. But, there are many others. Some of us may have experience with tab separated-value files, `TSV`.

Loading separated-value files is a common data extraction task that many experienced researchers are familiar and comfortable with. Often times, to execute  the code for data extraction takes milliseconds once you have the dataset's separated-value file downloaded on your computer for many common datasets we encounter in the social sciences.

Since CSV files are such a common file to extract our data from, let's start there. The CSV file I am working with in these examples is a dataset that has recorded the amount of Waffle Houses in a particular state and information about the rate of marriages and divorces in that same state. This is a relatively small dataset as it is $50\times13$ (50 rows and 13 columns). I've included this CSV file in the supplementary materials for the book online. For those interested in going straight to the source for the data, you should reference @mcelreath_statistical_2020.

### `Python`

In `Python` there are two dataframe management libraries that I will consider. Of course, there are many more than this, but here I want to focus on `Pandas` which is by far one of the most popular libraries in `Python`. I will also discuss `Polars` which is newer and not as adopted as `Pandas`. The benefit of looking into `Polars` is that it is extremely performant. The `Polars` library is a API for `Polars` which is written natively in `Rust` (an increasingly popular alternative to compiled languages such as `C++`).

Once I downloaded the dataset, I put it in my project folder called `data/`. I first want to specify the location of that particular file. It is not sufficient for me to just tell my computer to look for a file called `WaffleDivorce.csv` because it would take an extremely long time for my computer to look through millions of files for a matched name. The other problem of not being a bit more specific about the location of the file is that I may have multiple files on my computer called `WaffleDivorce.csv` but for other projects. If you have ever looked through your computer and have found multiple files called `Untitled1.docx`, this is a pretty common experience that many of us have. Just as you'd be confused if you need to open `Untitled1.docx` on your Desktop or in a specific folder somewhere else on your computer, so is your computer if you aren't a bit more specific about *where* on your computer to find the file. So, in this case, I am going to tell it to find the file in my current project (`./`) that is within my `data/` folder. So, the specific path to the CSV is `./data/WaffleDivorce.csv`. On line 2, you will see that I store this path for repeated use in an object called `csv_path`.

Once, I have specified the path to my file by placing it in an object called `csv_path`, I can then load the `Pandas` library by writing `import pandas` as you see on line 6. I have an extra step on that same line where I add `as pd` which allows me to load `Pandas` but to give it a nickname so I do not have to write out `pandas` to then access a function.

So as you see on line 7 where I load the csv file, I write `pd.read_csv`. Which means that I am using the function `read_csv` from the `pandas` library. But, since I had used `as pd`, I didn't have to write `pandas.read_csv`.

On line 8, I write the code that will load my CSV file. I tell `Python` to use the `read_csv` function from `Pandas` where I use my `csv_path` object as an argument to the function to tell it which file to load. I also specify another argument for the object. This argument is a named argument called `delimiter`. This argument allows me to specify what in the file defines separation between cells. In this case, the file is stored as a CSV file, but it actually uses semi-colons, `;`, as the separator rather than a comma, `,`. Under the hood, `Pandas` is taking the data from the `WaffleDivorce.csv` file and is extracting it and then converting it into a `Pandas` `DataFrame` object by keying in on the delimiter that I specify. There are many other arguments that you can pass to the `Pandas` `read_csv` function that helps you work with separated-value files that may have other strange characteristics, but I recommend you looking at `Pandas`' official documentation to explore your options. Once loaded and placed in a `Pandas` `DataFrame` object, the object is then linked to the object `anes_df` which allows me to refer back to the `Pandas` `DataFrame` later so that I do not have to re-extract the data later.


In [ ]:
#| label: python-load-waffle-house
#| eval: false
# Define location of csv file
csv_path='./data/2020-anes/anes.csv'

# Pandas
    #* Import Pandas
import pandas as pd
    #* Load CSV
anes_df = pd.read_csv(csv_path, delimiter=';')

# Polars
    #* Import Polars
import polars as pl
    #* Load CSV
anes_df = pl.read_csv(csv_path, separator=';')

# Preview of dataframe
anes_df.head()

I perform the same task but this time I extract the data and place it into a `Polars` `DataFrame` instead of pandas. On line 12, I import the `Polars` library and give it the nickname of `pl` similar to how I did with `pandas`. Notice how I do not give them the same nickname. I do this to be careful about confusing `Python` about whether I am referring to `Pandas` or `Polars`.

On line 14, I extract the data and place it into a `Polars` dataframe with the `read_csv` function from the `Polars` library. In this case, `Polars` and `Python` both call their function `read_csv` if you are loading a separated-values file and are trying to place it into one of their `DataFrames`. As a side note, this is a problem that common users of `R` face where many are often much less explicit about the particular function they want to use than you are in `Python`. But in `Python`, we know that the `read_csv` functions are different and should produce two different types of `DataFrame` objects, `Pandas` or `Polars`, because we append the nickname of the library to the function name (e.g. `pd.read_csv` versus `pl.read_csv`).

We see another difference on line 14. With `Polars`' `read_csv` function, I use the named argument `separator` as opposed to `delimiter`. These diverging choices in what to name these arguments for their `read_csv` functions are choices that the developers of the libraries chose. But, they do the same thing. The `separator` argument specifies what value separates the cells from each other. Which with the `WaffleDivorce.csv` file was a semi-colon, `;`.


In [ ]:
#| label: profile-python-waffle-house-load
#| include: false
# Define path for csv file
csv_path='../data/2020-anes/anes.csv'
# Pandas
    #* Define a function to be ran through
def pandas_extract_csv(path=csv_path):
    anes_df = pd.read_csv(path, delimiter=';')

    #* Repeat through the defined function 100 times and make list of time taken
pandas_load_time = timeit.repeat(
    'pandas_extract_csv()', # run the function
    'from __main__ import pandas_extract_csv', # after importing it
    number=1, # run through it once
    repeat=100 # but repeat this 100 times
)

# Polars
    #* Define a function to be ran through
def polars_extract_csv(path=csv_path):
    anes_df = pl.read_csv(path, separator=';')

    #* Repeat through the defined function 100 times and make a list of time taken
polars_load_time = timeit.repeat(
    'polars_extract_csv()', # run the function
    'from __main__ import polars_extract_csv', # after importing it
    number = 1, # run through it once
    repeat=100 # but repeat this 100 times
)

Keep in mind that this is the same file but I am extracting data from it with two different libraries and it produces two different types of `DataFrames`. The first is a `Pandas` `DataFrame` and the second is a `Polars` `DataFrame`. So, not only are the ways in which these packages extract the data different from each other but the resulting `DataFrames` are different. They are not different in that the contents of the tables are different, but rather the fundamentals of the table are different. As we will discuss in the chapters about data transformation, this has a pretty significant impact on how we think about transforming and interacting with our data in terms of what our code looks like and the performance of our code.

But, to focus on differences in the performance of the data extraction task, I wanted to examine the speed at which these two libraries load the file, transform the data into a tabular format, and then assign it to a callable `DataFrame` object. For this particular file, the speed at which this happens is extremely quick because we are dealing with a small amount of data (50 rows and 13 columns). However, when we work with larger datasets that can be thousands of rows and hundreds of columns, the speed at which this data extraction occurs can decrease significantly. Importantly, the differences between both of these packages can also diverge. That is, `Polars` claims to have smaller decreases in speed relative to `Pandas` as we are operating with larger datasets. These differences between file sizes and between libraries increases significantly the larger these datasets become. But for now, I look at this particular dataset and how quickly these two libraries can extract the data.


In [ ]:
#| output: asis
#| echo: false
print(
    f'I use the `timeit` package to calculate the average amount of time it took `Pandas` and `Polars` to load the WaffleDivorce.csv file 100 times. It took `Pandas` on average {mean(pandas_load_time):.2e} (Std. Deviation = {stdev(pandas_load_time):2e}) seconds while it took `Polars` an average of {mean(polars_load_time):.2e} (Std. Deviation = {stdev(polars_load_time):.2e}) seconds.'
)

I am able to independently replicate `Polars`' claims that their library loads data quicker than `Pandas` does. The average differences in speed are extremely small. The results of this profiling (test of how long a set of code took to execute) are reported in scientific notation. However, as mentioned before, as we have more data we should expect the time it takes to load these files will increase for both `Pandas` and `Polars`, as well as the differences in the time between the two libraries. But demonstrating that latter comparison will be a task for the next chapter. Let us look at data extraction in `R`.

### `R`


```{r}
#| label: r-waffle-house-load
#| eval: false
# Define path to CSV
csv_path <- './data/2020-anes/anes.csv'
# Base R
    #* Load CSV file
anes_df <- read.csv(csv_path, sep=';')

# Tidyverse
    #* Load readr
library(readr)
    #* Load CSV file
anes_df <- read_delim(csv_path, delim=';')

# Preview dataframe
head(anes_df)
```

```{r}
#| label: profile-r-waffle-house-load
#| include: false
# Define path to csv file
csv_path <- './data/2020-anes/anes.csv'

# Base R
    #* Clear tictoc log
tic.clearlog()
    #* Profiling each step of this process
for(i in 1:100) { # repeat the following 100 times
    tic(i) # start timer
    anes_df <- read.csv(csv_path, sep=';') # load the csv
    toc(i, quiet=TRUE) # stop timer
}
base_r_csv_benchmark_log <- tic.log(format=FALSE) # grab the log of how long this took
base_r_csv_benchmark <- unlist( # and split this log up
    lapply(
        base_r_csv_benchmark_log # by repeatedly taking the entries in the log
        , function(x) x$toc - x$tic # and subtracting the start time from the stop time for each iteration
    )
)

# Tidyverse
    #* Clear tic toc log
tic.clearlog()
    #* Profiling each step of this process
for(i in 1:100) { # repeat the following 100 times
    tic(i) # start timer
    anes_df <- read_delim(csv_path, delim=';', show_col_types=FALSE) # load the file time
    toc(i, quiet=TRUE) # stop timer
}
tidyverse_csv_benchmark_log <- tic.log(format=FALSE) # grab the log of how long this took
tidyverse_csv_benchmark <- unlist( # and split this log up
    lapply(
        tidyverse_csv_benchmark_log # by repeatedly taking the entries in the log
        , function(x) x$toc - x$tic # and subtracting the start time from the stop time for each iteration
    )
)
```

```{r}
#| output: asis
#| echo: false
cat(
    "I use the `tictoc` package to calculate the average amount of time it took `Base R` and the `Tidyverse` package to extract data from the anes.csv file and to generate a `DataFrame` 100 times. It took `Base R` an average of ", format(mean(base_r_csv_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(base_r_csv_benchmark), digits=3, scientific=TRUE), ") seconds per attempt, while it took `Tidyverse` an average of ", format(mean(tidyverse_csv_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(tidyverse_csv_benchmark), digits=3, scientific=TRUE), ") seconds per attempt."
    , sep=""
)
```


### `DuckDB`


In [ ]:
#| label: sql-load-waffle-house
#| eval: false
# Import the DuckDB library
import duckdb as db

# Use the Python API for DuckDB to connect to the database
con = db.connect('./data/2020-anes/anes.db')

# Use the connection to extract the data from the csv and place it as a table with the DuckDB API
con.execute(# Execute the following SQL query ...
    '''
        CREATE OR REPLACE TABLE -- create or replace a table
            main -- ... called main
        AS -- and place the following data in the main table
            SELECT -- select the following columns
                *  -- all of them
            FROM -- from ...
                read_csv_auto('./data/2020-anes/anes.csv') -- this specific file
        /*
            NOTE:
                - I am using a function by DuckDB called read_csv_auto 
                  that extracts the data for me. There are some
                  alternatives available if you use PostgreSQL or MySQL.
                - I specify the file path inside the read_csv_auto function.
                  When I do this, I use single quotation marks.
        */
    '''
)

In [ ]:
#| label: sql-profile-waffle-house-load
#| include: false
# Polars
    #* Define a function to be ran through
def duckdb_extract_anes():
    con.execute(# Execute the following SQL query ...
        '''
            CREATE OR REPLACE TABLE -- create or replace a table
                main -- ... called main
            AS -- with the following data
                SELECT -- select the following columns
                    * -- all of them
                FROM -- from ...
                    read_csv_auto('../data/2020-anes/anes.csv') -- this specific file
            /*
                NOTE:
                    - I am using a function by DuckDB called read_csv_auto 
                      that extracts the data for me. There are some
                      alternatives available if you use PostgreSQL or MySQL.
                    - I specify the file path inside the read_csv_auto function.
                      When I do this, I use single quotation marks.
            */
        '''
    )
    #* Repeat the defined function 100 times and make a list of time taken
duckdb_load_time = timeit.repeat(
    'duckdb_extract_anes()', # run this function
    'from __main__ import duckdb_extract_anes', # after importing it
    number = 1, # and run the function once
    repeat = 100 # but repeat this process 100 times
)

In [ ]:
#| output: asis
#| echo: false
print(
    f"I use the `timeit` package to calculate the average amount of time it took the `DuckDB` API in `Python`  to load the main table from the WaffleDivorce.DB 100 times. It took `DuckDB` on average {mean(duckdb_load_time):.2e} (Std. Deviation = {stdev(duckdb_load_time):.2e}) seconds."
)

## Extracting data from Parquet files

### `Python`


In [ ]:
#| label: python-load-parquet-anes
#| eval: false
# Specify location of Parquet file
parquet_path='./data/2020-anes/anes.parquet'

# Pandas
    #* Load pandas
import pandas as pd
    #* Extract the data from parquet file and place it in a Pandas DataFrame
anes_df = pd.read_parquet(parquet_path)

# Polars
    #* Load polars
import polars as pl
    #* Extract data from parquet file and place it in a Polars DataFrame
anes_df = pl.read_parquet(parquet_path)

In [ ]:
#| label: python-profile-load-parquet-anes
#| include: false
# Specify location of parquet file
parquet_path='../data/2020-anes/anes.parquet'

# Pandas
    #* Define the function to run through
def pandas_parquet_extract_anes(path=parquet_path):
    anes_df = pd.read_parquet(path)
    #* Run the defined function repeatedly and keep track of how long it takes
pandas_parquet_anes_time = timeit.repeat(
    'pandas_parquet_extract_anes()', # execute the function
    'from __main__ import pandas_parquet_extract_anes', # after loading it
    number=1, # and run this once
    repeat=100 # but run this process 100 times
)

# Polars
    #* Define the function to run through
def polars_parquet_extract_anes(path=parquet_path):
    anes_dff = pl.read_parquet(path)
    #* Run the defined function repeatedly and keep track of how long it takes
polars_parquet_anes_time = timeit.repeat(
    'polars_parquet_extract_anes()', # execute the function
    'from __main__ import polars_parquet_extract_anes', # after loading it
    number=1, # and fun this once
    repeat=100 # but run this process 100 times
)

In [ ]:
#| output: asis
#| echo: false
print(
    f"I use the `timeit` package to calculate the average amount of time it took `Pandas` and `Polars` in `Python` to load the parquet file 100 times. It took `Pandas` on average {mean(pandas_parquet_anes_time):.2e} (Std. Deviation = {stdev(pandas_parquet_anes_time):.2e}) seconds and `Polars` an average of {mean(polars_parquet_anes_time):.2e} (Std. Deviation = {stdev(polars_parquet_anes_time):.2e}) seconds."
)

### `R`


```{r}
#| label: r-extract-parquet
#| eval: false
# Define path to the parquet file
parquet_path <- './data/2020-anes/anes.parquet'

# Base R
    #* Load the arrow package
library(arrow)
    #* Extract the data from the parquet file and place it in a Tidyverse DataFrame
anes_df <- read_arrow(parquet_path)
    #* Convert to a Base R DataFrame
anes_df <- as.data.frame(anes_df)

# Tidyverse
    #* Load arrow package
library(arrow)
    #* Extract data from parquet file
anes_df <- read_arrow(parquet_path)

# Preview of dataframe
head(anes_df)
```

```{r}
#| label: r-profile-extract-parquet
#| include: false
# Define path to the parquet file
parquet_path <- './data/2020-anes/anes.parquet'

# Base R
    #* Clear tic toc log
tic.clearlog()
    #* Profiling each step of this process
for(i in 1:100){ # repeat the following 100 times
    tic(i) # start timer
    anes_df <- read_parquet(parquet_path) # load the file
    anes_df <- as.data.frame(anes_df) # convert it to a Base R DataFrame
    toc(i, quiet=TRUE) # stop timer
}
base_r_parquet_benchmark_log <- tic.log(format=FALSE) # log how long this loop took
base_r_parquet_benchmark <- unlist( # split up the log
    lapply( # and for each iteration from the log, do the following
        base_r_parquet_benchmark_log
        , function(x) x$toc-x$tic # calculate the difference between the start and stop time
    )
)

# Tidyverse
    #* Clear tic toc log
tic.clearlog()
    #* Profiling each step of this process
for(i in 1:100) { # repeat the following 100 times
    tic(i) # start timer
    anes_df <- read_parquet(parquet_path) # load the file
    toc(i, quiet=TRUE) # stop timer
}
tidyverse_parquet_benchmark_log <- tic.log(format=FALSE) # log how long this loop took
tidyverse_parquet_benchmark <- unlist( # split up the log
    lapply( # and for each iteration from the log, do the following
        tidyverse_parquet_benchmark_log
        , function(x) x$toc - x$tic # calculate the difference between the start and stop time
    )
)
```

```{r}
#| output: asis
#| echo: false
cat(
    "I use the `tictoc` package to calculate the average amount of time it took the `Arrow` package to extract data from the anes.csv file and to generate a `Tidyverse` DataFrame as well as a `Base R` `DataFrame` 100 times. To produce a `Base R` `DataFrame` it took `Arrow` an average of ", format(mean(base_r_parquet_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(base_r_parquet_benchmark), digits=3, scientific=TRUE), ") seconds per attempt, while it took `Arrow` an average of ", format(mean(tidyverse_parquet_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(tidyverse_parquet_benchmark), digits=3, scientific=TRUE), ") seconds per attempt to produce a `Tidyverse` `DataFrame`."
    , sep=""
)
```


### `DuckDB`


In [ ]:
#| label: duckdb-parquet-extract-anes
#| eval: false
# Load duckdb library
import duckdb as db

# Connect to database
con = db.connect('./data/2020-anes/anes.db')

# Create a SQL table from the parquet file
con.execute(
    '''
        CREATE OR REPLACE TABLE -- create or replace a table
            main -- ... called main
        AS -- with the following data
            SELECT -- select the following columns
                * -- all of them
            FROM -- from
                read_parquet('./data/2020-anes/anes.parquet'); -- the anes.parquet file
        /*
            NOTE:
                - I am using the read_parquet function provided by DuckDB
                - I am specifying the file inside single quotations within the function.
        */
    '''
)

In [ ]:
#| label: duckdb-profile-extract-parquet
#| include: false
# Profile Parquet extraction by duckdb
    #* Define function
def duckdb_parquet_extract_anes(path=parquet_path):
    con.execute(
        '''
            CREATE OR REPLACE TABLE -- create or replace a table
            main -- ... called main
            AS -- with the following data 
                SELECT -- select the following columns
                    * -- all of them
                FROM -- from
                    read_parquet('./data/2020-anes/anes.parquet'); -- the anes.parquet file
            /*
                NOTE:
                    - I am using the read_parquet function provided by DuckDB
                    - I am specifying the file inside single quotations within the function.
            */
        '''
    )

    #* Profile how long it takes to do it 100 times
duckdb_parquet_anes_time = timeit.repeat(
    'duckdb_parquet_extract_anes()', # run this function
    'from __main__ import duckdb_parquet_extract_anes', # after loading it
    number=1, # and run it once
    repeat=100 # but run this whole process 100 times
)

## Extracting data from Proprietary file types (i.e., XLSX, DTA, SAV)

### XLSX Files

#### `Python`


In [ ]:
#| label: python-extracting-xlsx
#| eval: false
# Define the path to the xlsx file
xlsx_path = './data/2020-anes/anes.xlsx'

# Pandas
    #* Load library
import pandas as pd
    #* Extract data from the xlsx file
anes_df = pd.read_excel(xlsx_path)

# Polars
    #* Load library
import polars as pl
    #* Extract data from the xlsx file
anes_df = pl.read_excel(xlsx_path)

# Display a preview of the data.frame
anes_df.head()

In [ ]:
#| label: profile-python-extracting-xlsx
#| include: false
# Define the path to the xlsx file
xlsx_path = '../data/2020-anes/anes.xlsx'

# Pandas
    #* Define function to run through
def pandas_extract_xlsx(path=xlsx_path):
    anes_df = pd.read_excel(path)

    #* Profile how long it takes to run 100 times
pandas_extract_xlsx_time = timeit.repeat(
    'pandas_extract_xlsx()', # run the function
    'from __main__ import pandas_extract_xlsx', # after loading it
    number=1, # and run it once
    repeat=100 # but run this whole process 100 times
)

# Polars
    #* Define function to run through
def polars_extract_xlsx(path=xlsx_path):
    anes_df = pl.read_excel(path)

    #* Profile how long it takes to run 100 times
polars_extract_xlsx_time = timeit.repeat(
    'polars_extract_xlsx()', # run the function
    'from __main__ import polars_extract_xlsx', # after loading it
    number=1, # run this once
    repeat=100 # but repeat this whole process 100 times
)

In [ ]:
#| output: asis
#| echo: false
# Print out the descriptive statistics of how muc time it took
print(
    f"I use the `timeit` package to calculate the average amount of time it took `Pandas` and `Polars` to extract the file and to generate a `Pandas` and `Polars` `DataFrame` object, respectively. `Pandas` took an average of {mean(pandas_extract_xlsx_time):.2e} (Std. Deviation = {sd(pandas_extract_xlsx_time):.2e}) seconds per attempt, while it took `Polars` an average of {mean(polars_extract_xlsx_time):.2e} (Std. Deviation = {sd(polars_extract_xlsx_time):.2e}) seconds."
)

#### `R`


```{r}
#| label: r-load-xlsx
#| eval: false
# Define the path to xlsx file
xlsx_path <- './data/2020-anes/anes.xlsx'

# Base R
    #* Import the readxl package
library(readxl)
    #* Extract the data and store as a Tidyverse DataFrame
anes_df <- read_excel(xlsx_path)
    #* Then convert it to a Base R DataFrame
anes_df <- as.data.frame(anes_df)

# Tidyverse
    #* Import package
library(readxl)
    #* Load file
anes_df <- read_excel(xlsx_path)
```

```{r}
#| label: r-profile-extract-parquet
#| include: false
# Define the path to the xlsx file
xlsx_path <- './data/2020-anes/anes.xlsx'

# Base R
    #* Clear tictoc log
tic.clearlog()
    #* Profiling each step of this process
for(i in 1:100){ # repeat the following 100 times
    tic(i) # start timer
    anes_df <- read_excel(xlsx_path) # load the file
    anes_df <- as.data.frame(anes_df) # convert it to a Base R DataFrame
    toc(i, quiet=TRUE) # stop timer
}
base_r_xlsx_benchmark_log <- tic.log(format=FALSE) # log how long this loop took
base_r_xlsx_benchmark <- unlist(# split the log up
    lapply(# and do the following to each iteration in the log
        base_r_xlsx_benchmark_log
        , function(x) x$toc - x$tic # take the difference between the start and stop time
    )
)

# Tidyverse
    #* Clear tic toc log
tic.clearlog()
    #* Profiling each step of this process
for(i in 1:100) { # repeat the following 100 times
    tic(i) # start timer
    anes_df <- read_excel(xlsx_path) # load the file
    toc(i, quiet=TRUE) # stop timer
}
tidyverse_xlsx_benchmark_log <- tic.log(format=FALSE) # log how long this loop took
tidyverse_xlsx_benchmark <- unlist( # split the log up
    lapply( # and do the following to each iteration in the log
        tidyverse_xlsx_benchmark_log
        , function(x) x$toc - x$tic # take the difference between the start and stop time
    )
)
```

```{r}
#| output: asis
#| echo: false
cat(
    "I use the `tictoc` package to calculate the average amount of time it took the `readxl` package to extract data from the anes.xlsx file and to generate a `Tidyverse` `DataFrame` as well as a `Base R` `DataFrame` 100 times. To produce a `Base R` `DataFrame`, it took `readxl` an average of ", format(mean(base_r_xlsx_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(base_r_xlsx_benchmark), digits=3, scientific=TRUE), ") seconds per attempt, while it took `readxl` to produce a `Tidyverse` `DataFrame` an average of ", format(mean(tidyverse_xlsx_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(tidyverse_xlsx_benchmark), digits=3, scientific=TRUE), ") seconds per attempt."
    , sep=""
)
```


#### `DuckDB`


In [ ]:
#| label: duckdb-extract-xlsx
#| eval: false
# Import the duckdb library
import duckdb as db

# Connect to the database with DuckDB's Python API
con = db.connect('./data/2020-anes/anes.db')

# Need to install and load the Spatial extension to help with reading the excel file
con.execute(
    '''
        INSTALL spatial -- only need to run this once per DuckDB connection.
        LOAD spatial -- only need to run this once per DuckDB connection.
    '''
)

# Create a table called main in the database with the extracted data from the xlsx file
con.execute(
    '''
        CREATE OR REPLACE TABLE -- create or replace a table
            main -- ... called main
        AS 
            SELECT -- select the following columns
                * -- all of them
            FROM -- from
                st_read('./data/2020-anes/anes.xlsx', layer='Sheet1'); -- the first sheet in the anes.xlsx file
        /*
            NOTE:
              - Using the st_read function provided by the Spatial extension to read the excel file.
              - Specify the file inside single quotation marks.
              - Need to specify which sheet I should be extracting the data from.
        */
    '''
)

In [ ]:
#| label: duckdb-profile-extract-xlsx
#| include: false
# Profile how long it takes to extract data from the xlsx file and generate a table 100 times
    #* Define the function to run through
def duckdb_extract_xlsx():
    con.execute(
        '''
            CREATE OR REPLACE TABLE -- create or replace a table
                main -- ... called main
            AS 
                SELECT -- select the following columns
                    * -- all of them
                FROM -- from
                    st_read('./data/2020-anes/anes.xlsx', layer='Sheet1'); -- the first sheet in the anes.xlsx file
            /*
                NOTE:
                  - Using the st_read function provided by the Spatial extension to read the excel file.
                  - Specify the file inside single quotation marks.
                  - Need to specify which sheet I should be extracting the data from.
            */
        '''
    )
    #* Document how long it takes to run the following 100 times
duckdb_extract_xlsx_time = timeit.repeat(
    'duckdb_extract_xlsx()', # run the function
    'from __main__ import duckdb_extract_xlsx', # after loading the function
    number=1, # do this once
    repeat=100 # then repeat this process 100 times
)

In [ ]:
#| output: asis
#| echo: false
# Print out the descriptive statistics of how much time it took
print(
    f"I use the `timeit` package to calculate the average amount of time it took `DuckDB` a table. `DuckDB` took an average of {mean(duckdb_extract_xlsx_time):.2e} (Std. Deviation = {sd(duckdb_extract_xlsx_time):.2e}) seconds per attempt."
)

### DTA Files

#### `Python`


In [ ]:
#| label: python-extract-dta
#| eval: false
# Define path to dta file
dta_path = './data/2020-anes/anes.dta'

# Pandas
    #* Load pandas library
import pandas as pd
    #* Extract data from file and place in Pandas DataFrame
anes_df = pd.read_stata(dta_path)

# Polars
    #* Load pandas and polars library
import polars as pl
import pandas as pd
    #* Extract the data from the file and place in Pandas DataFrame
anes_df = pd.read_stata(dta_path)
    #* Convert Pandas DataFrame to Polars DataFrame
anes_df = pl.from_pandas(anes_df)

# Preview the DataFrame
anes_df.head()

In [ ]:
#| label: python-profile-extract-dta
#| include: false
# Define path to dta file
dta_path = '../data/2020-anes/anes.dta'

# Pandas
    #* Define function to run through
def pandas_extract_dta(path=dta_path):
    anes_df = pd.read_stata(dta_path)

    #* Profile how long each iteration takes
pandas_extract_dta_time = timeit.repeat(
    'pandas_extract_dta()', # run this function
    'from __main__ import pandas_extract_dta', # after importing the function
    number=1, # read it once
    repeat=100 # and repeat this whole process 100 times
)

# Polars
    #* Define function to run through
def polars_extract_dta(path=dta_path):
    anes_df = pd.read_stata(dta_path)
    anes_df = pl.from_pandas(anes_df)

    #* Profile how long each iteration takes
polars_extract_dta_time = timeit.repeat(
    'polars_extract_dta()', # run thi function
    'from __main__ import polars_extract_dta', # after importing the function
    number=1, # read it once
    repeat=100 # and repeat the whole process 100 times
)

In [ ]:
#| output: asis
#| echo: false
# Print out the descriptive statistics of how muc time it took
print(
    f"I use the `timeit` package to calculate the average amount of time it took `Pandas` and `Polars` to extract the file and to generate a `Pandas` and `Polars` `DataFrame` object, respectively. `Pandas` took an average of {mean(pandas_extract_dta_time):.2e} (Std. Deviation = {sd(pandas_extract_dta_time):.2e}) seconds per attempt, while it took `Polars` an average of {mean(polars_extract_dta_time):.2e} (Std. Deviation = {sd(polars_extract_dta_time):.2e}) seconds."
)

#### `R`


```{r}
#| label: r-extract-dta
#| eval: false
# Define path to dta file
dta_path <- './data/2020-anes/anes.dta'

# Base R
    #* Load the haven package (foreign package only supports up to STATA 12)
library(haven)
    #* Extract the dta file and create a Tidyverse DataFrame
anes_df <- read_dta(dta_path)
    #* Convert the Tidyverse DataFrame into a Base R DataFrame
anes_df <- as.data.frame(anes_df) 

# Tidyverse
    #* Load haven package
library(haven)
    #* Extract the dta file and create a Tidyverse DataFrame
anes_df <- read_dta(dta_path)

# Preview DataFrame
head(anes_df)
```

```{r}
#| label: r-profile-extract-dta
#| include: false
# Define path to dta file
dta_path <- './data/2020-anes/anes.dta'

# Base R
    #* Clear tictoc log
tic.clearlog()
    #* Profile how long it takes to load the dta file 100 times
for(i in 1:100) {
    tic(i) # start the timer at each start of an iteration
    anes_df <- read_dta(dta_path) # extract the dta file and place it in a Tidyverse DataFrame
    anes_df <- as.data.frame(anes_df) # convert the Tidyverse DataFrame to a Base R DataFrame
    toc(i, quiet=TRUE) # stop timer
}
base_r_dta_benchmark_log <- tic.log(format=FALSE) # log how long this loop took
base_r_dta_benchmark <- unlist( # split the log file by iteration
    lapply(# for each iteration, do the following
    base_r_dta_benchmark_log,
    function(x) x$toc - x$tic # calculate the difference between the stop and start time
    )
)

# Tidyverse
    #* Clear tictoc log
tic.clearlog()
    #* Profile how long it takes to load the dta file 100 times.
for(i in 1:100) {
    tic(i) # start timer at each start of an iteration
    anes_df <- read_dta(dta_path) # extract the data from the dta file and place in a DataFrame
    toc(i, quiet=TRUE) # stop timer
}
tidyverse_dta_benchmark_log <- tic.log(format=FALSE) # log how long this loop took
tidyverse_dta_benchmark <- unlist(lapply(tidyverse_dta_benchmark_log, function(x) x$toc - x$tic))
```

```{r}
#| output: asis
#| echo: false
cat(
    "I use the `tictoc` package to calculate the average amount of time it took the `haven` package to extract data from the anes.dta file and to generate a `Tidyverse` `DataFrame` as well as a `Base R` `DataFrame` 100 times. To produce a `Base R` `DataFrame`, it took `haven` an average of ", format(mean(base_r_dta_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(base_r_dta_benchmark), digits=3, scientific=TRUE), ") seconds per attempt, while it took `haven` to produce a `Tidyverse` `DataFrame` an average of ", format(mean(tidyverse_dta_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(tidyverse_dta_benchmark), digits=3, scientific=TRUE), ") seconds per attempt."
    , sep=""
)
```


#### `DuckDB`


In [ ]:
#| label: duckdb-extract-dta
#| eval: false
# Define path to dta file
dta_path = './data/2020-anes/anes.dta'
# Load duckdb library
import duckdb as db
# Connect to database
con = db.connect('./data/2020-anes/anes.db')

# Load DTA file into Pandas DataFrame
    #* Load pandas library
import pandas as pd
    #* Load the file
anes_df = pd.read_stata(dta_path, convert_categoricals=False)

# Then create a table object out of the Pandas DataFrame
con.execute(
    '''
        CREATE OR REPLACE TABLE -- create a table
            main -- ... called main
        AS SELECT -- using the columns
            * -- specifically all of them
        FROM
            anes_df -- from the anes_df pandas DataFrame
    '''
)

In [ ]:
#| label: duckdb-profile-extract dta
#| include: false
# Define path to dta file
dta_path = '../data/2020-anes/anes.dta'

# Define function to run through
def duckdb_extract_dta(path=dta_path):
    #* Load anes.dta into a Pandas DataFrame
    anes_df = pd.read_stata(path, convert_categoricals=False)
    #* Then create a table object out of the Pandas DataFrame
    con.execute(
        '''
            CREATE OR REPLACE TABLE -- create a table
                main -- ... called main
            AS SELECT -- using the columns
                * -- specifically all of them
            FROM
                anes_df -- from the anes_df pandas DataFrame
        '''   
    )
# See how long it takes to go through this function each time 100 times.
duckdb_extract_dta_time = timeit.repeat('duckdb_extract_dta()', 'from __main__ import duckdb_extract_dta', number=1, repeat=100)

In [ ]:
#| output: asis
#| echo: false
# Print out the descriptive statistics of how muc time it took
print(
    f"I use the `timeit` package to calculate the average amount of time it took a combination of `Pandas` and `DuckDB` to extract the file and to generate a table. `Pandas` and `DuckDB` took an average of {mean(duckdb_extract_dta_time):.2e} (Std. Deviation = {sd(duckdb_extract_dta_time):.2e}) seconds per attempt."
)

### SAV Files

#### `Python`


In [ ]:
#| label: python-extract-sav
#| eval: false
# Define path of SAV file
sav_path = './data/2020-anes/anes.sav'

# Pandas
    #* Import pandas
import pandas as pd
    #* Load file
anes_df = pd.read_spss(sav_path)

# Polars
    #* Import pandas and polars
import polars as pl
import pandas as pd
    #* Extract data from the file and store it as a pandas DataFrame
anes_df = pd.read_spss(sav_path)
    #* Convert pandas DataFrame to polars DataFrame
anes_df = pl.from_pandas(anes_df)

In [ ]:
#| label: python-profile-extract-sav
#| include: false
# Define path of SAV file
sav_path = '../data/2020-anes/anes.sav'

# Pandas
    #* Define function to run through
def pandas_extract_sav(path=sav_path):
    anes_df = pd.read_spss(path)

    #* Profile how long it takes to run through this function 100 times
pandas_extract_sav_time = timeit.repeat(
    'pandas_extract_sav()', # run the function
    'from __main__ import pandas_extract_sav', # after loading it
    number=1, # run it one time
    repeat=100 # but repeat the whole process 100 times
)

# Polars
    #* Define the function to run through
def polars_extract_sav(path=sav_path):
    anes_df = pd.read_spss(path)
    anes_df = pl.from_pandas(anes_df)

    #* Profile how long it takes to run through this function 100 times
polars_extract_sav_time = timeit.repeat(
    'polars_extract_sav()', # run the function
    'from __main__ import polars_extract_sav', # after loading it
    number=1, # run it one time
    repeat=100 # but repeat the whole process 100 times
)

In [ ]:
#| output: asis
#| echo: false
# Print out the descriptive statistics of how muc time it took
print(
    f"I use the `timeit` package to calculate the average amount of time it took `Pandas` and `Polars` to extract the file and to generate a `Pandas` and `Polars` `DataFrame` object, respectively. `Pandas` took an average of {mean(pandas_extract_sav_time):.2e} (Std. Deviation = {sd(pandas_extract_sav_time):.2e}) seconds per attempt, while it took `Polars` an average of {mean(polars_extract_sav_time):.2e} (Std. Deviation = {sd(polars_extract_sav_time):.2e}) seconds."
)

#### `R`


```{r}
#| label: r-load-sav-file
#| eval: false
# Define file path to SAV file
sav_path <- './data/2020-anes/anes.sav'

# Base R
    #* Load foreign library
library(foreign)
    #* Extract the data and place it into a Base R DataFrame
anes_df <- read.spss(sav_path, to.data.frame=TRUE)

# Tidyverse
    #* Load haven library
library(haven)
    #* Extract the data and place it into a Tidyverse DataFrame
anes_df <- read_spss(sav_path)
```

```{r}
#| label: r-profile-extract-sav
#| include: false
# Define file path to SAV file
sav_path <- '../data/2020-anes/anes.sav'

# Clear tictoc log
tic.clearlog()

# Base R
    # Profile how long it takes to load the sav file 100 times
for(i in 1:100) {
    tic(i) # start timer
    anes_df <- read.spss(sav_path, to.data.frame=TRUE) # extract the data and place it in a DataFrame
    toc(i, quiet=TRUE) # stop the timer
}
base_r_sav_benchmark_log <- tic.log(format=FALSE) # log how long this loop took
base_r_sav_benchmark <- unlist( # split the log into the iterations I ran
    lapply( # and perform the following on each iteration
        base_r_sav_benchmark_log
        , function(x) x$toc - x$tic # calculate the difference between the stop and start times
    )
)

# Clear tictoc log
tic.clearlog()

# Tidyverse
    # Profile how long it takes to load the sav file 100 times
for(i in 1:100) {
    tic(i) # start timer
    anes_df <- read.spss(sav_path, to.data.frame=TRUE) # extract the data and place it in a DataFrame
    toc(i, quiet=TRUE) # stop the timer
}
tidyverse_sav_benchmark_log <- tic.log(format=FALSE) # log how long this loop took
tidyverse_sav_benchmark <- unlist(# split the log into the iterations I ran
    lapply( # and perform the following on each iteration
        tidyverse_sav_benchmark_log
        , function(x) x$toc - x$tic # calculate the difference between the stop and start times
    )
)
```

```{r}
#| output: asis
#| echo: false
cat(
    "I use the `tictoc` package to calculate the average amount of time it took the `foreign` and `haven` packages to extract data from the anes.dta file and to generate a `Base R` and `Tidyverse` `DataFrame` 100 times. To produce a `Base R` `DataFrame`, it took `foreign` an average of ", format(mean(base_r_sav_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(base_r_sav_benchmark), digits=3, scientific=TRUE), ") seconds per attempt, while it took `haven` to produce a `Tidyverse` `DataFrame` an average of ", format(mean(tidyverse_sav_benchmark), digits=3, scientific=TRUE), " (Std. Deviation = ", format(sd(tidyverse_sav_benchmark), digits=3, scientific=TRUE), ") seconds per attempt."
    , sep=""
)
```


#### `DuckDB`


In [ ]:
#| label: duckdb-extract-sav
#| eval: false
# Define path to sav file
sav_path = './data/2020-anes/anes.sav'

# Import duckdb library
import duckdb as db

# Open connection to the database
con = db.connect('./data/2020-anes/anes.db')

# Extract the data with Pandas and place in a Pandas DataFrame
anes_df = pd.read_spss(
    sav_path, 
    convert_categoricals=False # do not try to keep the factor encoding for the columns
)

# Place the Pandas DataFrame in a table
con.execute(
    '''
        CREATE OR REPLACE TABLE -- create or replace a table
            main -- called main
        AS -- using the data from the following query
            SELECT -- select the following columns
                * -- all of them
            FROM
                anes_df -- from the anes_df Pandas DataFrame
    '''
)

In [ ]:
#| label: duckdb-profile-extract-sav
#| include: false
# Define path to sav file
sav_path = '../data/2020-anes/anes.sav'

# Define function to run through
def duckdb_extract_sav(path=sav_path):
    #* Extract data from sav file and place in pandas DataFrame
    anes_df = pd.read_spss(
        path, 
        convert_categoricals=False # do not try to keep the factor encoding for the columns
    )
    # Place the Pandas DataFrame in a table
    con.execute(
        '''
            CREATE OR REPLACE TABLE -- create or replace a table
                main -- called main
            AS -- using the data from the following query
                SELECT -- select the following columns
                    * -- all of them
                FROM
                    anes_df -- from the anes_df Pandas DataFrame
        '''
    )
# Run the function 100 times
duckdb_extract_sav_time = timeit.repeat(
    'duckdb_extract_sav()', # run this function
    'from __main__ import duckdb_extract_sav', # after loading the function
    number=1, # run this 1 time
    repeat=100 # but repeat the whole process 100 times
)

In [ ]:
#| output: asis
#| echo: false
# Print out the descriptive statistics of how muc time it took
print(
    f"I use the `timeit` package to calculate the average amount of time it took a combination of `Pandas` and `DuckDB` to extract the file and to generate a table. `Pandas` and `DuckDB` took an average of {mean(duckdb_extract_dta_time):.2e} (Std. Deviation = {sd(duckdb_extract_dta_time):.2e}) seconds per attempt."
)

## Comparing performance across libraries, languages, and file types


In [ ]:
 #| label: close-db-connection
#| include: false
# Close the duckdb connection
con.close()